### Make neut plots of all six monoclonal antibodies used for mapping escape in Nipah RBP

In [ ]:
# this cell is tagged as parameters for `papermill` parameterization
altair_config = None
mab_neuts = None
mab_neuts_plot = None

In [ ]:
import warnings
import math

from IPython.display import display, HTML, SVG

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import neutcurve
from neutcurve.colorschemes import CBPALETTE
from neutcurve.colorschemes import CBMARKERS
import scipy.stats

from plotnine import *

import yaml

import altair as alt
import re
import os

print(f"Using `neutcurve` version {neutcurve.__version__}")

In [ ]:
# allow more rows for Altair
_ = alt.data_transformers.disable_max_rows()

if os.getcwd() == '/fh/fast/bloom_j/computational_notebooks/blarsen/2023/Nipah_Malaysia_RBP_DMS/':
    pass
    print("Already in correct directory")
else:
    os.chdir("/fh/fast/bloom_j/computational_notebooks/blarsen/2023/Nipah_Malaysia_RBP_DMS/")
    print("Setup in correct directory")

In [ ]:
#altair_config = 'data/custom_analyses_data/theme.py'
#nipah_config = 'nipah_config.yaml'
#mab_neuts = 'data/custom_analyses_data/experimental_data/231220_mAb_all_redo.csv'
#mab_neuts_plot = 

In [ ]:
if altair_config:
    with open(altair_config, 'r') as file:
        exec(file.read())

In [ ]:
#load data
df = pd.read_csv(mab_neuts)

In [ ]:
fits = neutcurve.curvefits.CurveFits(
            data=df,
            fixbottom=0,
            )
fitParams = fits.fitParams(ics=[50, 90, 95, 97, 98, 99])
fitParams['ic50_ng'] = fitParams['ic50'] * 1000
display(fitParams[['serum','ic50_ng']])

In [ ]:
def extract_dataframe_from_neutcurve(serum, viruses, replicate='average'):
    curves = []
    # Loop over each virus type and retrieve the curve
    for sera in serum:
        #print(sera)
        #print(viruses)
        curve = fits.getCurve(serum=sera, virus=viruses, replicate=replicate)
        df = curve.dataframe()
        df['antibody'] = sera
        curves.append(df)
    
    # Concatenate all the dataframes into one
    curve = pd.concat(curves, axis=0)
    curve['upper'] = curve['measurement'] + curve['stderr']
    curve['lower'] = curve['measurement'] - curve['stderr']
    
    return curve

# Example usage:
serum = ['nAH1.3','m102.4','HENV-26','HENV-32','HENV-103','HENV-117']
viruses = 'NiV'
curve = extract_dataframe_from_neutcurve(serum, viruses)
display(curve.head(3))

In [ ]:
def plot_neut():
    chart = alt.Chart(curve).mark_line(size=1.5,opacity=1).encode(
        x=alt.X('concentration:Q',scale=alt.Scale(type='log'),axis=alt.Axis(format='.0e'),title='Concentration (μg/mL)'),
        y=alt.Y('fit:Q',title='Fraction Infectivity'), 
        color=alt.Color('antibody',title='Antibody')
    ).properties(
        height=alt.Step(10),
        width=300,
    )

    circle = alt.Chart(curve).mark_circle(size=40,opacity=1).encode(
        x=alt.X('concentration',scale=alt.Scale(type='log'),axis=alt.Axis(format='.0e'),title='Concentration (μg/mL)'),
        y=alt.Y('measurement:Q',title='Fraction Infectivity'), 
        color=alt.Color('antibody',title='Antibody')
    ).properties(
        height=alt.Step(10),
        width=300,
    )

    error = alt.Chart(curve).mark_errorbar(opacity=1).encode(
        x='concentration',
        y=alt.Y('lower',title='Fraction Infectivity',axis=alt.Axis(tickCount=3)),
        y2='upper',
        color='antibody'
    )
    plot = chart+circle+error
    return plot

neut_chart = plot_neut()
neut_chart.display()
neut_chart.save(mab_neuts_plot)
